In [1]:
import psycopg2
import pandas as pd

In [15]:
def send_sql_query(query):
    params = {
        'database': 'focus_start',
        'host': 'localhost',
        'user': 'focus',
        'password': 'start'
    }
    conn = psycopg2.connect(**params)
    raw_data = pd.read_sql_query(query, conn)
    conn.close()
    return raw_data


In [16]:
tables = ['building_metadata','measurement_results', 'weather_train' ]

In [17]:
#1
send_sql_query("""
WITH meter_with_building AS (
SELECT  DISTINCT(meter) AS meter_1
       ,building_id
FROM measurement_results )
SELECT  COUNT(*)
FROM 
(
    SELECT  building_id
    FROM meter_with_building
    GROUP BY  building_id
    HAVING COUNT(meter_1) > 1
) AS result
""")

,count
0,572


In [18]:
#2
send_sql_query("""
SELECT  A.*
       ,B.*
       ,C.site_id
       ,C.air_temperature
       ,C.cloud_coverage
       ,C.dew_temperature
       ,C.precip_depth_1_hr
       ,C.sea_level_pressure
       ,C.wind_direction
       ,C.wind_speed
FROM 
(measurement_results A
    LEFT JOIN building_metadata B
    ON A.building_id = B.building_id
)
LEFT JOIN weather_train C
ON B.site_id = C.site_id AND A.timestamp_measurement = C.timestamp_measurement
WHERE A.timestamp_measurement > '2016-12-30' 
""")

,building_id,meter,timestamp_measurement,meter_reading,site_id,building_id,primary_use,square_feet,year_built,floor_count,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,131,0,2016-12-30 01:00:00,178.1710,1,131,Lodging/residential,66661,1930.0,7.0,1.0,0.1,9.0,0.1,NaN,1037.3,350.0,0.5
1,132,0,2016-12-30 01:00:00,33.0000,1,132,Lodging/residential,83108,1995.0,8.0,1.0,0.1,9.0,0.1,NaN,1037.3,350.0,0.5
2,133,0,2016-12-30 01:00:00,54.0000,1,133,Lodging/residential,64723,1960.0,8.0,1.0,0.1,9.0,0.1,NaN,1037.3,350.0,0.5
3,134,0,2016-12-30 01:00:00,83.0000,1,134,Lodging/residential,49589,1998.0,8.0,1.0,0.1,9.0,0.1,NaN,1037.3,350.0,0.5
4,135,0,2016-12-30 01:00:00,32.6000,1,135,Lodging/residential,66532,1967.0,10.0,1.0,0.1,9.0,0.1,NaN,1037.3,350.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111028,286,1,2016-12-30 01:00:00,35.2037,2,286,Lodging/residential,32841,1935.0,NaN,2.0,21.7,2.0,4.4,0.0,1018.9,120.0,1.5
111029,287,0,2016-12-30 01:00:00,24.2100,2,287,Office,33713,1956.0,NaN,2.0,21.7,2.0,4.4,0.0,1018.9,120.0,1.5
111030,287,1,2016-12-30 01:00:00,1.5474,2,287,Office,33713,1956.0,NaN,2.0,21.7,2.0,4.4,0.0,1018.9,120.0,1.5
111031,287,3,2016-12-30 01:00:00,0.0000,2,287,Office,33713,1956.0,NaN,2.0,21.7,2.0,4.4,0.0,1018.9,120.0,1.5


In [19]:
%%time
#3
send_sql_query("""
 WITH main_table AS (
SELECT  building_id AS bid
       ,meter
       ,meter_reading
FROM measurement_results), feature_1_table AS (
SELECT  building_id
       ,LOG(square_feet) AS logfeet
FROM building_metadata), feature_2_table AS (
SELECT  building_metadata.building_id
       ,2017.0 - year_built AS age
FROM building_metadata)
SELECT  bid
       ,meter
       ,meter_reading
       ,logfeet
       ,age
FROM main_table
LEFT JOIN feature_1_table AS f1
ON main_table.bid = f1.building_id
LEFT JOIN feature_2_table AS f2
ON main_table.bid = f2.building_id
""")

Wall time: 7.1 s


,bid,meter,meter_reading,logfeet,age
0,1363,2,156.392,5.198473,85.0
1,1364,0,38.750,4.874609,77.0
2,1364,2,1135.660,4.874609,77.0
3,1365,0,238.875,5.112819,76.0
4,1366,0,39.100,4.944369,70.0
...,...,...,...,...,...
2019672,1444,0,8.750,4.292677,103.0
2019673,1445,0,4.825,3.633266,NaN
2019674,1446,0,0.000,4.051731,20.0
2019675,1447,0,159.575,4.473852,16.0


In [20]:
#4
send_sql_query("""
SELECT  building_id
       ,meter
       ,meter_reading
       ,SUM(meter_reading) OVER(PARTITION BY building_id,meter ORDER BY timestamp_measurement) AS mr_cumsum
FROM measurement_results
""")

,building_id,meter,meter_reading,mr_cumsum
0,0,0,209.546,209.54600
1,0,0,220.467,430.01300
2,0,0,234.801,664.81396
3,0,0,228.658,893.47200
4,0,0,233.436,1126.90800
...,...,...,...,...
2019672,1448,0,3.400,2667.75270
2019673,1448,0,2.700,2670.45260
2019674,1448,0,3.050,2673.50270
2019675,1448,0,3.125,2676.62770


In [21]:
#5

send_sql_query("""
SELECT  building_id
       ,meter
       ,timestamp_measurement
FROM 
(
    SELECT  building_id
           ,meter
           ,timestamp_measurement
           ,ROW_NUMBER() OVER (PARTITION BY building_id,meter ORDER BY timestamp_measurement DESC) rn
    FROM measurement_results 
) AS tmp
WHERE tmp.rn = 1  
""")

,building_id,meter,timestamp_measurement
0,0,0,2016-12-31 23:00:00
1,1,0,2016-12-31 23:00:00
2,2,0,2016-12-31 23:00:00
3,3,0,2016-12-31 23:00:00
4,4,0,2016-12-31 23:00:00
...,...,...,...
2369,1444,0,2016-12-31 23:00:00
2370,1445,0,2016-12-31 23:00:00
2371,1446,0,2016-12-31 23:00:00
2372,1447,0,2016-12-31 23:00:00
